In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# Analysis based on downloaed data in ../data folder

In [ ]:


def analyze_stock(ticker: str, start_year: str, selected_year: str = None):
    """
    Analyzes stock data for a given ticker and start year.

    Parameters:
    ticker (str): The stock ticker symbol.
    start_year (str): The start year for the analysis.
    selected_year (str): The year from which to start the analysis. If None, analyze all data.

    Returns:
    None
    """
    # Load the CSV file, skipping the first 3 rows
    csv_filepath = f'../data/{ticker}_{start_year}.csv'
    df = pd.read_csv(csv_filepath, skiprows=3)

    # Manually set column names since the header is skipped
    df.columns = ['Date', 'Close', 'High', 'Low', 'Open', 'Volume']

    # Select only the 'Date' and 'Close' columns
    df = df[['Date', 'Close']]

    # Parse dates from the 'Date' column
    df['Date'] = pd.to_datetime(df['Date'])

    # Filter the DataFrame if selected_year is provided
    if selected_year is not None:
        df = df[df['Date'].dt.year >= int(selected_year)]

    # Set 'Date' as the index
    df.set_index('Date', inplace=True)

    # Calculate daily percentage changes for Close prices
    df['Pct_Change'] = df['Close'].pct_change() * 100
    df['Pct_Change'] = df['Pct_Change'].round(2)  # Round to two decimal places

    # print size of df
    print(f"Dataframe size: {df.shape}")

    # Plot the Close Price
    plt.figure(figsize=(12, 6))
    plt.plot(df.index, df['Close'], label='Close Price')
    plt.xlabel('Date')
    plt.ylabel('Close Price')
    plt.title(f'{ticker} Close Price Over Time')
    plt.legend()
    plt.grid(True)
    plt.show()

    # Plot the Percentage Change
    plt.figure(figsize=(12, 6))
    plt.plot(df.index, df['Pct_Change'], label='Percentage Change', color='orange')
    plt.xlabel('Date')
    plt.ylabel('Percentage Change')
    plt.title(f'{ticker} Percentage Change Over Time')
    plt.legend()
    plt.grid(True)
    plt.show()

    # Save results to new CSV in the '../data' folder
    output_folder = '../data'
    os.makedirs(output_folder, exist_ok=True)
    output_filepath = os.path.join(output_folder, f'{ticker}_analysis_results.csv')
    df.to_csv(output_filepath)

    print(f"Analysis complete. Results saved to {output_filepath}")

# Example usage
analyze_stock('MCHI', '2023', '2024')

In [ ]:
# Example usage
analyze_stock('^HSI', '2023', '2024')

In [ ]:
# Example usage
analyze_stock('YINN', '2023', '2024')

# Load data with pct_change and merge them

In [ ]:
# Load the results CSV files
df1 = pd.read_csv('../data/MCHI_analysis_results.csv', parse_dates=['Date'], index_col='Date')
df2 = pd.read_csv('../data/^HSI_analysis_results.csv', parse_dates=['Date'], index_col='Date')

# Merge the two DataFrames on the 'Date' index
merged_df = df1[['Pct_Change']].merge(df2[['Pct_Change']], left_index=True, right_index=True, suffixes=('_MCHI', '_HSI'))

START_DATE = '2023-09-24'
# START_DATE = '2024-09-24'
# START_DATE = '2025-01-01'

# Filter the merged DataFrame to include only dates after START_DATE
merged_df = merged_df[merged_df.index >= START_DATE]

# reverse order on date
merged_df = merged_df.sort_index(ascending=False)

# print total number of rows
TOTAL_ROWS = len(merged_df)
print(f"Total number of rows: {len(merged_df)}")

# Find rows where the product of Pct_Change is negative
negative_product_df = merged_df[(merged_df['Pct_Change_MCHI'] * merged_df['Pct_Change_HSI']) < 0]

# Find rows where the product of Pct_Change is non-negative
non_negative_product_df = merged_df[(merged_df['Pct_Change_MCHI'] * merged_df['Pct_Change_HSI']) >= 0]


In [ ]:

# Print the rows
print(negative_product_df)

In [ ]:
# Define the date range for plotting
start_date = '2024-12-01'
end_date = '2025-01-01'

# Filter the merged DataFrame to include only dates within the specified range
filtered_df = merged_df[(merged_df.index >= start_date) & (merged_df.index <= end_date)]

# Chart the filtered DataFrame
plt.figure(figsize=(12, 6))
plt.plot(filtered_df.index, filtered_df['Pct_Change_MCHI'], label='MCHI', color='blue')
plt.plot(filtered_df.index, filtered_df['Pct_Change_HSI'], label='HSI', color='red')
plt.xlabel('Date')
plt.ylabel('Percentage Change')
plt.title('MCHI vs HSI Percentage Change Over Time')
plt.legend()
plt.grid(True)
plt.show()

# plot negative ones

In [ ]:
# Define the date range for plotting
start_date = '2024-12-25'
end_date = '2025-03-01'
# print negative_product_df
# Filter the merged DataFrame to include only dates within the specified range
filtered_df = negative_product_df[(negative_product_df.index >= start_date) & (negative_product_df.index <= end_date)]  

# for each ticker, add column which is the sum of last two days percentage change for each ticker
filtered_df['Sum_Pct_Change_MCHI'] = filtered_df['Pct_Change_MCHI'].rolling(window=2).sum()
filtered_df['Sum_Pct_Change_HSI'] = filtered_df['Pct_Change_HSI'].rolling(window=2).sum()



# plot the filtered DataFrame
plt.figure(figsize=(12, 6))
plt.scatter(filtered_df.index, filtered_df['Pct_Change_MCHI'], label='MCHI', color='blue', s=100)
plt.scatter(filtered_df.index, filtered_df['Pct_Change_HSI'], label='HSI', color='red', s=100)
plt.axhline(y=0.0, color='black', linewidth=2)  # Add a thicker horizontal line at y=0.0
plt.xlabel('Date')
plt.ylabel('Percentage Change')
plt.title('MCHI vs HSI Percentage Change Over Time')
plt.legend()
plt.grid(True)
plt.show()

# show in datatable
# print(negative_product_df)


# print(negative_product_df)
negative_product_df

## Two days

In [ ]:
# plot filtered_df['Sum_Pct_Change_HSI'] and filtered_df['Sum_Pct_Change_MCHI']
plt.figure(figsize=(12, 6))
plt.plot(filtered_df.index, filtered_df['Sum_Pct_Change_MCHI'], label='MCHI', color='blue')
plt.plot(filtered_df.index, filtered_df['Sum_Pct_Change_HSI'], label='HSI', color='red')
plt.xlabel('Date')
plt.ylabel('Percentage Change')
plt.title('MCHI vs HSI Percentage Change Over Time')
plt.legend()
plt.grid(True)
plt.show()


## Ratio

In [ ]:
# print START_DATE
print(START_DATE)
# print len of negative_product_df
len(negative_product_df)/ len(merged_df) * 100

In [ ]:


# Print the rows
print(non_negative_product_df)

# TODO
- gap in gains
- hold with more days stratgies